In [1]:
from tqdm import tqdm
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import json
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import spacy
nlp = spacy.load('fr_core_news_md')

In [156]:
import stanza

In [3]:
nltk.download('stopwords')
stop_words = set(stopwords.words('french'))
stop_words.add("n'")
stop_words.add("l'")
stop_words.add("d'")
stop_words.add("c'")
stop_words.add("qu'")
stop_words.add("quelqu'")
stop_words.add("quelqu")
stop_words.add("s'")
stop_words.add("n’")
stop_words.add("l’")
stop_words.add("d’")
stop_words.add("c’")
stop_words.add("qu’")
stop_words.add("quelqu’")
stop_words.add("s’")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shaya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
json_file_path = 'stop_words_french.json'

# Open the JSON file
with open(json_file_path, encoding='utf-8') as json_file:
    # Load the JSON data
    stopwords = json.load(json_file)

In [5]:
stopwords = set(stopwords).union(stop_words)

In [6]:
with open('memoirs_preprocessed.pkl', 'rb') as f:
    data = pickle.load(f)

In [7]:
def extract_woi_context(commune_memoirs, wois, method, count = 0):

    woi_context_extraction = pd.DataFrame(columns=['filename', 'memoir_len_sans_sb_pb', 'bias', 'woi', 'woi_location', 'extraction_method', 'text'])

    for i, row in tqdm(commune_memoirs.iterrows()):

        filename = row[0]

        # commented out because this will corrupt lengths by token count
        #if filename == "du_camp_1.txt" or filename == "du_camp_2.txt":
        #    filename = "du_camp.txt"
        #if filename == "arnould_1.txt" or filename == "arnould_2.txt" or filename == "arnould_3.txt":
        #    filename = "arnould.txt"
        #if filename == "cluseret_1.txt" or filename == "cluseret_2.txt" or filename == "cluseret_3.txt":
        #    filename = "cluseret.txt"
        #if filename == "da_costa_1.txt" or filename == "da_costa_2.txt" or filename == "da_costa_3.txt":
        #    filename = "da_costa.txt"

        bias = row[1]
        text = row[2]
        
        memoir_len_sans_sb_pb = len([token for token in text.split() if token not in ["<sb>","<pb>"]])

        text_lower = text.lower()
        for woi in wois:
            if woi.lower() in text_lower:
                indices_object = re.finditer(pattern=r'\b{}\b'.format(re.escape(woi.lower())), string=text_lower)
                indices = [index.start() for index in indices_object]
                for location in indices:
                    if method == "token" or method == "character":
                        method_s = str(method) + ", " + str(count) + " either side"
                        window_half = count
                        if method == "token":
                            pre_tokens = ' '.join(text_lower[:location].split()[-window_half:])
                            post_tokens = ' '.join(text_lower[location:].split()[:window_half+len(woi.split())])
                            tmp_sequence = pre_tokens + " " + post_tokens
                            woi_location = len(pre_tokens) + 1
                        if method == "character":
                            pre_string = text_lower[:location][-window_half:]
                            post_string = text_lower[location:][:window_half+len(woi)]
                            tmp_sequence = pre_string + post_string
                            woi_location = 0
                            woi_location = len(pre_string)
                    elif method == "<sb>" or method == "<pb>":
                        method_s = method
                        pre_string = text_lower[text_lower[:location].rfind(method):location]
                        if "<pb>" in pre_string and method == "<sb>":
                            pre_string = pre_string[pre_string.rfind("<pb>"):]
                        post_string_tmp = text_lower[location:]
                        post_string = post_string_tmp[:post_string_tmp.find(method) + 4]
                        if "<pb>" in post_string and method == "<sb>":
                            post_string = post_string[:post_string.find("<pb>")]
                        tmp_sequence = pre_string + post_string
                        woi_location = 0
                        woi_location = len(pre_string)
                    woi_context_extraction.loc[len(woi_context_extraction)] = [filename, memoir_len_sans_sb_pb, bias, woi, woi_location, method_s, tmp_sequence]
    
    return woi_context_extraction

In [8]:
def tfidf_words(words, num_words = 10, max_freq = 0.8):
    
    df = extract_woi_context(data, words, "<sb>")
    df = df.drop_duplicates(subset='text', keep='first')
    df.text = df.text.str.replace('<pb>', '')
    df.text = df.text.str.replace('<sb>', '')
    df.text = df.text.apply(lambda t : nlp(t))
    df.text = df.text.apply(lambda i :[token.text for token in i if token.text not in stopwords])
    
    anti = df[df.bias == 'anti']
    pro = df[df.bias == 'pro']
    
    pro_text = ' '.join([word for sublist in list(pro.text.values) for word in sublist])
    anti_text = ' '.join([word for sublist in list(anti.text.values) for word in sublist])
    
    documents = [pro_text, anti_text]
    doc_type = ['pro', 'anti']
    
    # Create an instance of TfidfVectorizer
    vectorizer = TfidfVectorizer(stop_words = stopwords, max_df = max_freq)

    # Fit the vectorizer to the documents and calculate the TF-IDF values
    tfidf_matrix = vectorizer.fit_transform(documents)

    # Get the feature names (words)
    feature_names = vectorizer.get_feature_names_out()

    # Iterate over each document's TF-IDF values
    for doc_index, doc in enumerate(documents):
        # Get the sparse vector representation of the document
        sparse_vector = tfidf_matrix[doc_index]
        # Convert the sparse vector to a dense array
        dense_vector = sparse_vector.toarray()[0]
        # Create a list of (word, TF-IDF) tuples
        word_tfidf_list = [(feature_names[i], dense_vector[i]) for i in range(len(feature_names))]
        # Sort the list by TF-IDF values in descending order
        word_tfidf_list.sort(key=lambda x: x[1], reverse=True)
        # Print the top 10 words with the highest TF-IDF values for the document
        print(f"{doc_type[doc_index]}:")
        for word, tfidf in word_tfidf_list[:num_words]:
            print(f"{word}: {tfidf}")
        print()

In [9]:
with open('names.pkl', 'rb') as pickle_file:
    names = pickle.load(pickle_file)

In [10]:
names

['Raoul Rigault', 'Jules Favre', 'Garde Nationale', 'Félix Pyat', 'La Commune']

In [11]:
names = ['rigault', 'favre', 'pyat', 'ferré', 'commune', 'garde nationale']

In [12]:
for name in names:
    print(name)
    tfidf_words([name])

rigault


31it [00:02, 14.56it/s]


pro:
costa: 0.3048867566557235
révolutionnaire: 0.179345150973955
blanquistes: 0.107607090584373
chardon: 0.107607090584373
clermont: 0.107607090584373
jeune: 0.107607090584373
régnard: 0.107607090584373
tolérance: 0.107607090584373
blanquiste: 0.0896725754869775
bureaux: 0.0896725754869775

anti:
dacosta: 0.25801314467114733
aller: 0.12900657233557367
civil: 0.12900657233557367
obtenir: 0.11467250874273216
digne: 0.10033844514989064
présence: 0.10033844514989064
maître: 0.0860043815570491
mise: 0.0860043815570491
souvent: 0.0860043815570491
vit: 0.0860043815570491

favre


31it [00:01, 25.73it/s]


pro:
ans: 0.19133442763511838
faire: 0.1711939615682638
maître: 0.16112372853483653
assemblée: 0.15105349550140926
république: 0.15105349550140926
charmont: 0.13091302943455466
faussaire: 0.13091302943455466
jeanne: 0.13091302943455466
monsieur: 0.11077256336770011
vernier: 0.11077256336770011

anti:
prêtres: 0.2809179948882496
montmartre: 0.21068849616618718
cacheux: 0.175573746805156
préfecture: 0.175573746805156
assassiné: 0.1404589974441248
bandits: 0.1404589974441248
carrières: 0.1404589974441248
courbant: 0.1404589974441248
fermée: 0.1404589974441248
ignorantins: 0.1404589974441248

pyat


...

pro:
juges: 0.26592899812581355
fortin: 0.21606731097722348
déportation: 0.1495850614457701
enceinte: 0.13296449906290678
ailleurs: 0.11634393668004343
présence: 0.11634393668004343
blanquistes: 0.09972337429718008
dereure: 0.09972337429718008
tridon: 0.09972337429718008
cabinet: 0.08310281191431673

anti:
fédérés: 0.19668431441216472
brigadier: 0.13767902008851532
liberté: 0.13767902008851532
michel: 0.13767902008851532
ferait: 0.09834215720608236
foutre: 0.09834215720608236
individu: 0.09834215720608236
presque: 0.09834215720608236
ramain: 0.09834215720608236
yeux: 0.09834215720608236

commune


31it [00:11,  2.78it/s]


pro:
costa: 0.15627751460683262
blanquistes: 0.12651036896743592
re: 0.11906858255758676
nou: 0.07441786409849173
obligation: 0.07441786409849173
industriels: 0.06697607768864255
nouméa: 0.06697607768864255
andrieu: 0.05953429127879338
dois: 0.05953429127879338
financière: 0.05953429127879338

anti:
santé: 0.09787170074501608
siége: 0.09034310838001486
150: 0.06775733128501114
abattre: 0.06775733128501114
foutre: 0.06775733128501114
oeuvres: 0.06775733128501114
artistes: 0.060228738920009905
canonnières: 0.060228738920009905
chapelle: 0.060228738920009905
chevaux: 0.060228738920009905

garde nationale


31it [00:02, 13.85it/s]


pro:
propriété: 0.09951911611494307
pût: 0.09951911611494307
armés: 0.0870792266005752
bruit: 0.07463933708620729
cas: 0.07463933708620729
communales: 0.07463933708620729
père: 0.07463933708620729
rester: 0.07463933708620729
suprême: 0.07463933708620729
tion: 0.07463933708620729

anti:
commandants: 0.12295849095024002
mercredi: 0.12295849095024002
affiches: 0.10539299224306285
siége: 0.10539299224306285
ant: 0.08782749353588572
billioray: 0.08782749353588572
dépôt: 0.08782749353588572
respecter: 0.08782749353588572
régulièrement: 0.08782749353588572
60: 0.07026199482870857



In [13]:
garde = extract_woi_context(data, ['garde nationale'], "<sb>")

31it [00:02, 13.94it/s]


In [14]:
filtered_df = garde[garde['text'].str.contains('commandants')]
list(filtered_df.text.values)

["<pb> des mandats d'amener, nous dit-on, ont été lancés par le comité central contre les commandants des 17e, 18e, 19e, 20e, et 106e bataillons de la garde nationale, coupables d'être à la tête de bataillons composés en majorité d'honnêtes gens. <sb>",
 "<sb> deux commandants de la garde nationale, piazza et brunel, dans la nuit du 27 au 28 janvier, prirent, l’un le titre de général et l’autre celui de chef d'état-major et firent afficher dans paris un ordre du jour à la garde nationale. ",
 "<sb> deux commandants de la garde nationale, piazza et brunel, dans la nuit du 27 au 28 janvier, prirent, l’un le titre de général et l’autre celui de chef d'état-major et firent afficher dans paris un ordre du jour à la garde nationale. ",
 "<sb> on obéissait à tout le monde, au gouverneur, aux ministres, aux maires, aux chefs de corps, aux commandants de la garde nationale, aux présidents des comités et des clubs; ces autorités multiples détruisaient l'autorité. <sb>",
 "<pb> a l'annonce de ce 

In [15]:
filtered_df

,filename,memoir_len_sans_sb_pb,bias,woi,woi_location,extraction_method,text
737,arsac.txt,141550,anti,garde nationale,149,<sb>,"<pb> des mandats d'amener, nous dit-on, ont ét..."
906,bourde.txt,88824,anti,garde nationale,28,<sb>,"<sb> deux commandants de la garde nationale, p..."
907,bourde.txt,88824,anti,garde nationale,221,<sb>,"<sb> deux commandants de la garde nationale, p..."
1006,du_camp_2.txt,116707,anti,garde nationale,119,<sb>,"<sb> on obéissait à tout le monde, au gouverne..."
1033,du_camp_2.txt,116707,anti,garde nationale,156,<sb>,<pb> a l'annonce de ce que l'on nommait l'armi...
1040,du_camp_2.txt,116707,anti,garde nationale,590,<sb>,"<sb> alphonse daudet, a donné, dans le style v..."
1097,fayard.txt,193091,anti,garde nationale,64,<sb>,<pb> aux chefs de bataillons et commandants de...
1112,fayard.txt,193091,anti,garde nationale,88,<sb>,<pb> le 1er mai une réunion de commandants nom...


In [16]:
favre = extract_woi_context(data, ['favre'], "<sb>")

31it [00:01, 23.79it/s]


In [17]:
filtered_df = favre[favre['text'].str.contains('prêtres')]

In [18]:
filtered_df

,filename,memoir_len_sans_sb_pb,bias,woi,woi_location,extraction_method,text
377,arsac.txt,141550,anti,favre,310,<sb>,"<sb> la curieuse pièce suivante, affichée sur ..."
384,catulle.txt,89053,anti,favre,281,<sb>,"<sb> hier, on lisait l'affiche suivante lacard..."
434,du_camp_2.txt,116707,anti,favre,257,<sb>,"<sb> le 19 avril, on afficha cette sanie sur l..."
473,rodrigues.txt,58363,anti,favre,280,<sb>,<sb> un spécimen du genre sur l'affiche apposé...


In [19]:
list(filtered_df.text.values)

["<sb> la curieuse pièce suivante, affichée sur les portes fermées de l'église, donne les motifs de ces odieuses arrestations : « attendu que les prêtres sont des bandits et que les églises sont des repaires où ils ont assassiné moralement les masses, en courbant la france sous la griffe des infâmes bonaparte, favre et trochu, le délégué civil des carrières près l'ex-préfecture de police ordonne que l'église de saint-pierre (montmartre) soit fermée, et décrète l'arrestation des prêtres et des ignorantins. ",
 "<sb> hier, on lisait l'affiche suivante lacardée sur les portes fermées de l'église, à montmartre : « attendu que les prêtres sont des bandits t que les églises sont des repaires où ils ont assassiné noralement les masses en courbant la france sous la riffe des infames bonaparte, favre et trochu, le élégué des carrières près l'ex-préfecture de police rdonne que l'église de saint-pierre (et non de cinqierres, cette fois) soit fermée et décrète l'arrestation des prêtres et des igno

In [20]:
tfidf_words(['prêtres'])

31it [00:01, 28.77it/s]


pro:
eglise: 0.12524485821702913
mouchards: 0.12524485821702913
révolutionnaires: 0.12524485821702913
socialisme: 0.12524485821702913
instruction: 0.09393364366277185
lieu: 0.09393364366277185
misérables: 0.09393364366277185
presse: 0.09393364366277185
public: 0.09393364366277185
sang: 0.09393364366277185

anti:
prison: 0.16590798971560275
faut: 0.14931719074404246
murs: 0.14931719074404246
fusillé: 0.1327263917724822
père: 0.1327263917724822
bandits: 0.09954479382936164
incarcérés: 0.09954479382936164
magistrats: 0.09954479382936164
montmartre: 0.09954479382936164
picpus: 0.09954479382936164



---------------------

In [21]:
words = ['semaine sanglante', 'thiers', 'versaillais', 'communards']

In [22]:
for word in words:
    print(word)
    tfidf_words([word])

semaine sanglante


31it [00:00, 46.37it/s]


pro:
partie: 0.27650063180466594
chapitre: 0.22120050544373276
commune: 0.16590037908279956
militaire: 0.16590037908279956
bataille: 0.11060025272186638
défense: 0.11060025272186638
faveur: 0.11060025272186638
heure: 0.11060025272186638
massacre: 0.11060025272186638
milliers: 0.11060025272186638

anti:
appellent: 0.2581988897471612
aussitôt: 0.2581988897471612
auteur: 0.2581988897471612
commence: 0.2581988897471612
communards: 0.2581988897471612
déjà: 0.2581988897471612
imagé: 0.2581988897471612
mémoire: 0.2581988897471612
nom: 0.2581988897471612
porte: 0.2581988897471612

thiers


31it [00:02, 11.60it/s]


pro:
prussiens: 0.14350946197048078
révolution: 0.119591218308734
ailleurs: 0.11161847042148507
massacre: 0.11161847042148507
etat: 0.10364572253423612
prise: 0.10364572253423612
agent: 0.0956729746469872
doute: 0.0956729746469872
louis: 0.08770022675973826
versaillais: 0.08770022675973826

anti:
collection: 0.1849000654084082
objets: 0.13867504905630615
fontaine: 0.11556254088025514
aimait: 0.0924500327042041
arrête: 0.0924500327042041
consulat: 0.0924500327042041
gamin: 0.0924500327042041
24: 0.06933752452815307
approche: 0.06933752452815307
bronzes: 0.06933752452815307

versaillais


31it [00:01, 17.70it/s]


pro:
prussiens: 0.17485620077729147
000: 0.13599926727122671
sang: 0.1262850338947105
thiers: 0.10685656714167813
vinoy: 0.10685656714167813
amis: 0.09714233376516193
assemblée: 0.09714233376516193
bataillons: 0.09714233376516193
yeux: 0.09714233376516193
immédiatement: 0.08742810038864574

anti:
criait: 0.1374124274823272
signal: 0.1374124274823272
conduite: 0.10992994198586176
garçon: 0.10992994198586176
tenir: 0.10992994198586176
envoient: 0.08244745648939632
levallois: 0.08244745648939632
malade: 0.08244745648939632
matinée: 0.08244745648939632
opéra: 0.08244745648939632

communards


31it [00:00, 33.63it/s]


pro:
bagne: 0.2101565081224354
condamnés: 0.18680578499772035
forçats: 0.16345506187300532
france: 0.11675361562357524
bord: 0.09340289249886018
brissac: 0.09340289249886018
grand: 0.09340289249886018
nord: 0.09340289249886018
prendre: 0.09340289249886018
surveillant: 0.09340289249886018

anti:
colonne: 0.11219363880101454
alle1: 0.07479575920067637
armée: 0.07479575920067637
barricade: 0.07479575920067637
cluseret: 0.07479575920067637
croire: 0.07479575920067637
empire: 0.07479575920067637
empressèrent: 0.07479575920067637
exécution: 0.07479575920067637
faits: 0.07479575920067637



In [150]:
versaillais = extract_woi_context(data, ['thiers'], "<sb>")
filtered_df = versaillais[versaillais['text'].str.contains('collection')]

31it [00:02, 12.21it/s]


In [146]:
filtered_df

,filename,memoir_len_sans_sb_pb,bias,woi,woi_location,extraction_method,text
1148,arsac.txt,141550,anti,thiers,170,<sb>,<sb> ji « sur la délibération approuvée du com...
1151,bourde.txt,88824,anti,thiers,5,<sb>,"<sb> thiers, mais il laissa faire fontaine et ..."
1613,fayard.txt,193091,anti,thiers,114,<sb>,<sb> je donne lecture d'une lettre du citoyen ...
1614,fayard.txt,193091,anti,thiers,203,<sb>,"<pb> » le citoyen fontaine, directeur des omai..."
1719,morel.txt,43102,anti,thiers,5,<sb>,<sb> thiers démoli sous la présisidence du dél...


In [151]:
list(filtered_df.text.values)

['<sb> thiers et se souvint alors de son ancien métier : « la collection thiers, dit-il, se compose de richesses bibliographiques pour la conservation desquelles je demande qu’on nomme une commission. <sb>',
 '<sb> thiers et se souvint alors de son ancien métier : « la collection thiers, dit-il, se compose de richesses bibliographiques pour la conservation desquelles je demande qu’on nomme une commission. <sb>',
 "<sb> thiers ne se bornait pas seulement à la destruction d'un immeuble de jolie apparence, construit sur un terrain spacieux, dans un des plus élégants quartiers de paris ; ce qui le touchait plus encore que cette perte, c'était la disparition des objets d'art et des précieuses collections qu'il y avait réunies. <sb>",
 '<sb> dans une séance qui se tint le 12 mai, le citoyen courbet avait dit à ses collègues « le sieur thiers a une collection de bronzes antiques je demande ce que je dois en faire? <sb>',
 "<sb> thiers eut l'idée de se ménager, derrière son cabinet de travail,

In [62]:
list(filtered_df.text.values)

["<sb> » effectivement, quelques heures après, le dialogue suivant s'engageait entre lc format politique et l'aidemajor, qui devait payer de sa.révocation sa constante bienveillance pour les condamnés communards « est-ce bien lu fin, cette fois? ",
 '<sb> dans les derniers temps, ces braves gens ne nous considéraient plus du tout comme des condamnés; au risque de passer eux-mêmes pour communards, ils nous recevaient fréquemment à leur table, lucipia, fortin et moi, –ce)a au grand scandale des gardes-chiourme ébahis et de quelques officiers qui en avaient le tempérament. <sb>',
 "<sb> en sa pauvre cervelle, où le fanatisme le disputait à la haine contre les hommes de la commune, surgit cette pensée généreuse : faire crever de soif ces crapules de communards ! n’avaient-ils pas tué l'archevêque de paris, blasphémé le saint nom de dieu, insulté la religion ! on n’en avait pas assez massacré, puisqu'il y en avait encore à bord, et le bon catholique omit d'alimenter le foudre! vainement, le

In [32]:
woi = extract_woi_context(data, ['ferré'], "<sb>")
filtered_df = woi[woi['text'].str.contains('foutre')]
filtered_df

31it [00:02, 12.76it/s]


,filename,memoir_len_sans_sb_pb,bias,woi,woi_location,extraction_method,text
496,fayard.txt,193091,anti,ferré,225,<sb>,<pb> et nom de dieu ! il n'a pas tort ! commen...
499,fayard.txt,193091,anti,ferré,202,<sb>,<pb> non foutre ! on les prend pour servir je ...
500,fayard.txt,193091,anti,ferré,31,<sb>,"<pb> eh bien! foutre ! citoyen ferré, le père ..."
502,fayard.txt,193091,anti,ferré,118,<sb>,<pb> et les jean-foutres ne sauront pas où en ...


In [33]:
list(filtered_df.text.values)

["<pb> et nom de dieu ! il n'a pas tort ! comment, foutre ! on conspire dans paris, et le comité de salut public se remue comme un diable dans un bénitier, parce qu'il sent qu'il faut avoir l'oeil, et'ce nom de dieu de citoyen ferré, délégué à la sûreté, et le citoyen rigault procureur de la commune ne foutent rien, 278 histoire de la commune et boivent chopine sans doute, . ",
 "<pb> non foutre ! on les prend pour servir je peuple ! et si l'on devient un jour roussin, il faut qu'on soit roussin de la révolution et qu'on le sauve ! ce n'est pas ainsi que tu la sauveras, citoyen ferré ! de 1871. ",
 "<pb> eh bien! foutre ! citoyen ferré, le père duchène dit que la première chose à faire, c'est de foutre aux portes des citoyennes qui ouvrent l'oeil et qui ne laissent pas les mouchardes vaquer à leurs affaires, comme de bonnes petites bourgeoises , qui n'ont en tête que leur pot-au feu, et qui ne passent pas leur temps à faire de la contrebande ! fous aux portes de là ville des citoyen-. 

In [48]:
tfidf_words(['comité de salut public'])

31it [00:01, 15.51it/s]


pro:
ailleurs: 0.14213381090373947
capital: 0.14213381090373947
raison: 0.14213381090373947
caractère: 0.1184448424197829
conditions: 0.1184448424197829
poste: 0.1184448424197829
élection: 0.1184448424197829
action: 0.09475587393582632
choses: 0.09475587393582632
convention: 0.09475587393582632

anti:
vota: 0.3821610563931606
art: 0.2605643566317004
démocratique: 0.10422574265268017
honneur: 0.10422574265268017
élément: 0.10422574265268017
complémentaires: 0.08685478554390015
décrets: 0.08685478554390015
lit: 0.08685478554390015
temps: 0.08685478554390015
administration: 0.06948382843512012



In [39]:
woi = extract_woi_context(data, ['salut public'], "<sb>")
filtered_df = woi[woi['woi'] == 'comité du salut public']
woi

31it [00:02, 12.77it/s]


,filename,memoir_len_sans_sb_pb,bias,woi,woi_location,extraction_method,text
0,andre_leo.txt,10365,pro,salut public,171,<sb>,"<sb> ce sont ces échappés de collège qui, n’ay..."
1,arnould_2.txt,43978,pro,salut public,30,<sb>,<pb> le vote sur le comité de salut public est...
2,arnould_2.txt,43978,pro,salut public,253,<sb>,<pb> en parlant de la minorité et de la majori...
3,arnould_2.txt,43978,pro,salut public,157,<sb>,"<pb> eh bien, ni le décret sur les otages, qua..."
4,arnould_2.txt,43978,pro,salut public,79,<sb>,<pb> — non pas que je mîsse mes idées personne...
...,...,...,...,...,...,...,...
540,rodrigues.txt,58363,anti,salut public,18,<sb>,"<pb> le comité de salut public, ant. <sb>"
541,rodrigues.txt,58363,anti,salut public,18,<sb>,<pb> le comité de salut public autorise les ch...
542,rodrigues.txt,58363,anti,salut public,29,<sb>,"<pb> le membre dzc comité du salut public, g. ..."
543,rodrigues.txt,58363,anti,salut public,18,<sb>,"<pb> le comité de salut public, installé dans ..."


In [44]:
woi = extract_woi_context(data, ['comité de salut public', 'comité du salut public'], "<sb>")
filtered_df = woi[woi['text'].str.contains('démocrat')]
list(filtered_df.text.values)

31it [00:01, 23.03it/s]


["<pb> « des piques, des haches, des sabres nus, fles couperets et des marteaux ; « la cité morne et silencieuse ; la police au foyer de famille, les opinions suspectées, les paroles écoutées, les larmes observées, les soupirs comptés, le silence épié, l'espionnage et les dénonciations ;- « les réquisitions inexorables, les emprunts forcés et progressifs, le papier monnaie déprécié ; « hs guerre civile, et l'étranger sur les frontières ; « les proconsùlats impitoyables, le comité de salut public, un comité suprême au cœur d'airain ; c voilà les fruits de la révolution dite démocratique et sociale. ",
 '<sb> proudhon le disait en 1849, — proudhon l’ami de delesluze et qui conspirait alors avec lui.— «ce qu’il faut, écrivait-1l, à la révolution démocratique et ” sociale (journal de delescluze), c’est une perpéuelle et fatigante agitation qui, éclatant tout à coup, se termine par la création d’un comité de salut public, où certains patriotes trouvent une occupation digne de‘leur génie. <s

In [43]:
filtered_df

,filename,memoir_len_sans_sb_pb,bias,woi,woi_location,extraction_method,text
180,arsac.txt,141550,anti,comité de salut public,476,<sb>,"<pb> « des piques, des haches, des sabres nus,..."
239,bourde.txt,88824,anti,comité de salut public,297,<sb>,"<sb> proudhon le disait en 1849, — proudhon l’..."
251,bourde.txt,88824,anti,comité de salut public,43,<sb>,<sb> » : il s’abstint dans le vote pour le com...
290,du_camp_1.txt,118984,anti,comité de salut public,188,<sb>,<sb> avant de se consacrer à ces futilités bon...
399,fayard.txt,193091,anti,comité de salut public,24,<sb>,<pb> considérant que le comité de salut public...
432,hennebert.txt,77304,anti,comité de salut public,528,<sb>,"<sb> une multitude déchaînée, armée, ivre de v..."


In [45]:
tfidf_words(['!'])

31it [00:01, 30.98it/s]


pro:
déportation: 0.1417985101042947
garde: 0.1417985101042947
guerre: 0.12407369634125787
mt: 0.12407369634125787
peine: 0.12407369634125787
tt: 0.12407369634125787
jour: 0.10634888257822102
jamais: 0.0886240688151842
mhe: 0.0886240688151842
nationale: 0.0886240688151842

anti:
canons: 0.55942688102005
rayés: 0.3846059807012843
siège: 0.20978508038251872
ouest: 0.10489254019125936
troupes: 0.10489254019125936
aller: 0.06992836012750625
boulevards: 0.06992836012750625
chevaux: 0.06992836012750625
châtillon: 0.06992836012750625
genève: 0.06992836012750625



In [46]:
woi = extract_woi_context(data, ['!'], "<sb>")
filtered_df = woi[woi['text'].str.contains('mt')]
list(filtered_df.text.values)

31it [00:01, 30.77it/s]


["<sb> ftmoy, le f<o, .pamtkm, pr~~em~ du <yom!ke- tmemt, .apures <'<î'ampëres, 7!t(eftcmf, jus- ttcc et ~atre paris. ",
 "<sb> ftmoy, le f<o, .pamtkm, pr~~em~ du <yom!ke- tmemt, .apures <'<î'ampëres, 7!t(eftcmf, jus- ttcc et ~atre paris. ",
 "<pb> « ~<<' de paris à préfet de police, général vtmo!ekë<'a~ef<d,/mmrtemt',prmtdemt du ~om~rmem~m~. ",
 "<sb> il représente le yomi!cfketm~mt insurrectionnel; moi, je suis mmë 6m<m«<!ok du <yomt~'methëm( )'ë<ym<te?';j'em nommé adjoint par un vote légal il ne sera pas touché à un cheveu de la tète du général chanzy et de m. <sb>",
 "<sb> (très bien tr~ bien v~m marques d'adhésion et (['approbation.) et il continue par cette calomnie « comment paris, qui afficherait aujourd'hui la ~t~~m<!erep)'~em!tom t'tt're seul et de se séparer de ce qu'il appelle la prov~'ce, les kunaux, comme on le dit .bfat;9/~pp<amc!tmemfmts); comment, messieurs, pari.pourrait-il soutenir un seul instant cette erreur politique, économique et sociale. ",
 "<sb> (très bien tr

In [47]:
filtered_df

,filename,memoir_len_sans_sb_pb,bias,woi,woi_location,extraction_method,text
25,da_costa_1.txt,122048,pro,!,44,<sb>,"<sb> ftmoy, le f<o, .pamtkm, pr~~em~ du <yom!k..."
26,da_costa_1.txt,122048,pro,!,79,<sb>,"<sb> ftmoy, le f<o, .pamtkm, pr~~em~ du <yom!k..."
27,da_costa_1.txt,122048,pro,!,53,<sb>,"<pb> « ~<<' de paris à préfet de police, génér..."
35,da_costa_1.txt,122048,pro,!,26,<sb>,<sb> il représente le yomi!cfketm~mt insurrect...
37,da_costa_1.txt,122048,pro,!,166,<sb>,<sb> (très bien tr~ bien v~m marques d'adhésio...
38,da_costa_1.txt,122048,pro,!,277,<sb>,<sb> (très bien tr~ bien v~m marques d'adhésio...
40,da_costa_1.txt,122048,pro,!,116,<sb>,<pb> [i n'en est pas moins vrai que les représ...
41,da_costa_1.txt,122048,pro,!,18,<sb>,"<sb> umité,~m'c«ed!ëpasmtf; 2° parce que, dès ..."
45,da_costa_1.txt,122048,pro,!,38,<sb>,"<pb> vo<httb:.16,765 .~mt)'e;l.\t7lmta!t(~ dub..."
59,da_costa_2.txt,62863,pro,!,12,<sb>,<pb> « le pj!é5ident(d6st~h(!mtfen'ë).–est-ce ...


In [83]:
extract_woi_context(data, ['floréal'], "<sb>")

31it [00:00, 31.26it/s]


,filename,memoir_len_sans_sb_pb,bias,woi,woi_location,extraction_method,text
0,beslay.txt,49273,pro,floréal,20,<sb>,"<pb> « paris, le 27 floréal an 79."
1,arsac.txt,141550,anti,floréal,20,<sb>,"<pb> « paris, le-16 floréal an 79."
2,arsac.txt,141550,anti,floréal,20,<sb>,"<pb> « paris, le 16 floréal an 79."
3,arsac.txt,141550,anti,floréal,17,<sb>,"<pb> « paris, 21 floréal an 79."
4,arsac.txt,141550,anti,floréal,29,<sb>,"<pb> « hùtfl-de-ville, le 24 floréal an 79. <sb>"
5,arsac.txt,141550,anti,floréal,30,<sb>,"<pb> « ilôtel-de-ville, le 25 floréal an 79."
6,arsac.txt,141550,anti,floréal,20,<sb>,"<pb> « paris, le 25 floréal an 79."
7,arsac.txt,141550,anti,floréal,29,<sb>,"<pb> « hôtet-de-ville, le 26 floréal an 79."
8,arsac.txt,141550,anti,floréal,87,<sb>,<pb> « quatre des coupables sont entre les mai...
9,arsac.txt,141550,anti,floréal,209,<sb>,<pb> « gardes nationaux! votre choix est fait ...


In [82]:
tfidf_words(['rouge'])

31it [00:02, 14.12it/s]


pro:
spectre: 0.18633101085572115
boulevards: 0.10351722825317843
jaune: 0.10351722825317843
pirate: 0.10351722825317843
simon: 0.10351722825317843
derniers: 0.08281378260254274
ferré: 0.08281378260254274
honte: 0.08281378260254274
maire: 0.08281378260254274
partie: 0.08281378260254274

anti:
parti: 0.12688825371490364
revolver: 0.12688825371490364
000: 0.11102722200054069
devait: 0.11102722200054069
loque: 0.11102722200054069
maison: 0.11102722200054069
palais: 0.11102722200054069
affiche: 0.09516619028617772
carnaval: 0.09516619028617772
1870: 0.07930515857181478



In [152]:
def recursive_find_adjs(root, sent):
        children = [w for w in sent.words if w.head == root.id]
        if not children:
            return []
        filtered_c = [w for w in children if w.deprel == "conj" and w.upos == "ADJ"]
        # Do not include an adjective if it is the parent of a noun to prevent
        results = [w for w in filtered_c if not any(sub.head == w.id and sub.upos == "NOUN" for sub in sent.words)]
        for w in children:
            results += recursive_find_adjs(w, sent)
        return results

def extract_noun_adj_pairs(text, stanz_nlp):
    text = text.replace("<sb>","").replace("<pb>","")
    doc = stanz_nlp(text)
    noun_adj_pairs = {}
    for sent in doc.sentences:
        nouns = [w for w in sent.words if w.upos == "NOUN"]
        for noun in nouns:
            cop_root = sent.words[noun.head-1]
            adjs = [cop_root] + recursive_find_adjs(cop_root, sent) if cop_root.upos == "ADJ" else []
            mod_adjs = [w for w in sent.words if w.head == noun.id and w.upos == "ADJ"]
            if mod_adjs:
                mod_adj = mod_adjs[0]
                adjs.extend([mod_adj] + recursive_find_adjs(mod_adj, sent))
            if adjs:
                unique_adjs = []
                unique_ids = set()
                for adj in adjs:
                    if adj.id not in unique_ids:
                        unique_adjs.append(adj)
                        unique_ids.add(adj.id)
                noun_adj_pairs[noun.text] = " ".join([adj.text for adj in unique_adjs])
    return noun_adj_pairs

In [158]:
stanz_nlp = stanza.Pipeline("fr")

2023-05-29 19:17:13 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-05-29 19:18:11 INFO: Loading these models for language: fr (French):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |
| ner       | wikiner |

2023-05-29 19:18:11 INFO: Using device: cpu
2023-05-29 19:18:11 INFO: Loading: tokenize
2023-05-29 19:18:11 INFO: Loading: mwt
2023-05-29 19:18:11 INFO: Loading: pos
2023-05-29 19:18:12 INFO: Loading: lemma
2023-05-29 19:18:12 INFO: Loading: depparse
2023-05-29 19:18:12 INFO: Loading: ner
2023-05-29 19:18:13 INFO: Done loading processors!


In [176]:
tqdm.pandas()

In [178]:
extract = extract_woi_context(data, ['garde nationale'], "<sb>")
pd.set_option('max_colwidth', 400)
extract["noun_adj_pairs"] = extract["text"].progress_apply(extract_noun_adj_pairs, args=(stanz_nlp,))

31it [00:03,  9.04it/s]
100%|██████████████████████████████████████████████████████████████████████████████| 1315/1315 [37:03<00:00,  1.69s/it]


In [187]:
pro = goncourt[goncourt.bias == 'pro']
anti = goncourt[goncourt.bias == 'anti']

In [188]:
pro_adj = pro.noun_adj_pairs.values
anti_adj = anti.noun_adj_pairs.values

In [189]:
p_adj = []
a_adj = []
# Itération à travers chaque dictionnaire
for dictionnaire in pro_adj:
    # Récupération des valeurs du dictionnaire courant
    valeurs = dictionnaire.items()
    # Ajout des valeurs à la liste
    p_adj.extend(valeurs)
    
for dictionnaire in anti_adj:
    # Récupération des valeurs du dictionnaire courant
    valeurs = dictionnaire.items()
    # Ajout des valeurs à la liste
    a_adj.extend(valeurs)


In [190]:
from collections import Counter

# Comptage des occurrences des éléments
p_compteur = Counter(p_adj)
a_compteur = Counter(a_adj)

# Obtention du top 10 des éléments les plus mentionnés
p_top_10 = p_compteur.most_common(10)
a_top_10 = a_compteur.most_common(10)
# Affichage du top 10
for element, count in p_top_10:
    print(element, count)
print()
for element, count in a_top_10:
    print(element, count)


('gardes', 'nationaux') 17
('comité', 'central') 17
('salut', 'public') 16
('comité', 'public') 14
('garde', 'nationale') 13
('Journal', 'officiel') 8
('cour', 'martiale') 8
('délégué', 'civil') 8
('directeur', 'général') 5
('sûreté', 'générale') 5


In [174]:
extract.to_pickle('thiers_adj.pkl')

In [184]:
goncourt = pd.read_pickle('goncourt_arsac_dip.pkl')